In [1]:
import torch
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
from torch import nn

In [59]:
# Put on GPU
device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

# selct a preset of images:

imgs_1 = noisy_imgs_1[:1]/255
imgs_2 = noisy_imgs_2[:10000]/255
print(imgs_1.shape)

torch.Size([1, 3, 32, 32])


In [3]:
input_x = imgs_1
print(input_x.shape)
print(imgs_1.shape)

unf_input = unfold(imgs_1, kernel_size=(5, 5))
print(unf_input.shape)

conv1 = nn.Conv2d(3, 10, kernel_size = 5, stride = 1)
output_x = conv1(input_x)
print(output_x.shape)

NameError: name 'imgs_1' is not defined

In [2]:
print(imgs_1.shape)
unf_input = unfold(imgs_1, kernel_size=(5, 5))
print(unf_input.shape)
Hout = (32 - 5)/1 + 1
Wout = (32 - 5)/1 + 1
print(Hout, Wout)

NameError: name 'imgs_1' is not defined

In [7]:
w = torch.randn(3, 3, 5, 5)

x = torch.ones(1,3,6,6)

convT1 = nn.ConvTranspose2d(3, 3, kernel_size = 5, stride = 1)

y = convT1(x)
print('x', x.shape)
print('y',y.shape)

x torch.Size([1, 3, 6, 6])
y torch.Size([1, 3, 10, 10])


In [12]:
#convtrans

x = torch.ones(1,10,6,6)
w = torch.randn(3, 10, 5, 5)


print(x.shape)
x_reshape = x.permute(1, 2, 3, 0).reshape(10, -1)
print('x',x_reshape.shape)
w_reshape = w.reshape(10, -1)
print('w', w_reshape.shape)
y_unfolded = w_reshape.t().matmul(x_reshape)
print('y unfolded calculate',y_unfolded.shape)

y_unfolded = y_unfolded.reshape(y_unfolded.shape[0], -1, x.shape[0])
print('y',y_unfolded.shape)
y_unfolded = y_unfolded.permute(2, 0, 1)
print('y',y_unfolded.shape)


print('y fin', y_unfolded.shape)

H_out = (6 - 1)*1+ (5 - 1) + 1
W_out = (6 - 1)*1 + (5- 1) + 1

ou_t = fold(y_unfolded, (H_out, W_out), kernel_size=(5,5), stride=1)
print(ou_t.shape)

"""
H_in = self.x.shape[2]
W_in = self.x.shape[3]


return fold(y_unfolded, (H_out, W_out), kernel_size=self.kernel_size, stride=self.stride) + self.bias.view(1, -1, 1, 1)

"""

torch.Size([1, 10, 6, 6])
x torch.Size([10, 36])
w torch.Size([10, 75])
y unfolded calculate torch.Size([75, 36])
y torch.Size([75, 36, 1])
y torch.Size([1, 75, 36])
y fin torch.Size([1, 75, 36])
torch.Size([1, 3, 10, 10])


'\nH_in = self.x.shape[2]\nW_in = self.x.shape[3]\n\n\nreturn fold(y_unfolded, (H_out, W_out), kernel_size=self.kernel_size, stride=self.stride) + self.bias.view(1, -1, 1, 1)\n\n'

In [ ]:
class Module ( object ) :
    def forward ( self , x ) :
        
        x = convolution(x)

    def backward ( self , * gradwrtoutput ) :

    

In [5]:
imgs_1 = torch.ones(1,3,6,6)
print(imgs_1.shape)
unf_input = unfold(imgs_1, kernel_size=(4,4))
print(unf_input.shape)

Hout = (6 - 4)/1 + 1
Wout = (6 - 4)/1 + 1

print(Hout,Wout)






torch.Size([1, 3, 6, 6])
torch.Size([1, 48, 9])
3.0 3.0


In [6]:
imgs_1 = torch.ones(1,3,6,6)

a = imgs_1.reshape(1,3,-1)
b = imgs_1

print(a.shape, b.shape)

torch.Size([1, 3, 36]) torch.Size([1, 3, 6, 6])


In [81]:
#Upsampling

print(imgs_1.shape)
imgs_1 = torch.ones(1,3,6,6)
w = torch.randn(3, 3, 2, 2)

unf_input = unfold(imgs_1, kernel_size=(2,2))
print(unf_input.shape)

out_unf = unf_input.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
#out_unf = (w.view(w.size(0), -1).t()).matmul(unf_input.transpose(1, 2)).transpose(1, 2)
print(unf_input.transpose(1, 2).shape)
print(w.view(w.size(0), -1).t().shape)

print('permute mode')
print(unf_input.transpose(1, 2).permute(0,2,1).shape)
print(w.view(w.size(0), -1).t().permute(1,0).shape)

print(unf_input.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).shape)
print(out_unf.shape)
#out = nn.functional.fold(out_unf,(5, 5),(1,1))
print(out.shape)

torch.Size([1, 3, 6, 6])
torch.Size([1, 12, 25])
torch.Size([1, 25, 12])
torch.Size([12, 3])
permute mode
torch.Size([1, 12, 25])
torch.Size([3, 12])
torch.Size([1, 25, 3])
torch.Size([1, 3, 25])
torch.Size([1, 3, 5, 5])


In [7]:
imgs_1 = torch.ones(1,3,6,6)
_,_,H,W = imgs_1.shape
print(H,W)

6 6


In [8]:
#convolution with no stride and kernel of 5

imgs_1 = torch.ones(1,3,6,6)
print(imgs_1.shape)


def convolution(x):

    w = torch.randn(10, 3, 2, 2)
    print('w',w.shape)
    
    unf_input = unfold(imgs_1, kernel_size=(2, 2))
    print('unf_input',unf_input.shape)
    out_unf = unf_input.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
    print('1',unf_input.transpose(1, 2).shape)
    print('2',w.view(w.size(0), -1).t().shape)
    print('out_unf ',out_unf.shape )
    out = nn.functional.fold(out_unf,(5, 5),(1,1))
    print('out',out.shape)
    
    return out

y = convolution(imgs_1)

#out_ = torch.nn.functional.fold(out_unf, (h_out, w_out), (1, 1))


torch.Size([1, 3, 6, 6])
w torch.Size([10, 3, 2, 2])
unf_input torch.Size([1, 12, 25])
1 torch.Size([1, 25, 12])
2 torch.Size([12, 10])
out_unf  torch.Size([1, 10, 25])
out torch.Size([1, 10, 5, 5])


In [ ]:
# Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)

inp = torch.randn(1, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))

In [ ]:
Sequential ( Conv ( stride 2) ,
            ReLU ,
            Conv ( stride 2) ,
            ReLU ,
            Upsampling ,
            ReLU ,
            Upsampling ,
            Sigmoid )

In [72]:
ac = torch.randn((1,3,2,2))
print(ac)
#ac_unfold = fold(ac.reshape(1,3,-1), (4, 4), (1, 1), dilation=(2, 2))
#print('unfold',ac_unfold.shape)
ac = ac.reshape(1,-1)
 

print(ac.shape)
#print(ac)
#ac = ac.permute(0,2,1)
w = torch.ones(1, 4)
print(w.shape)
print(w)
print(ac)

y = ac.T @ w
print(y)
#print(y.shape)
y_view = y.view(1,3,4,4)
y_reshape = y.reshape(1,3,4,4)

print(y_view.shape, y_reshape.shape)
print(y_view)
print(y_reshape)




tensor([[[[-0.2636,  0.4540],
          [ 0.4969, -0.7056]],

         [[-0.7837,  1.8423],
          [ 0.5035,  0.3321]],

         [[-0.4469, -0.0372],
          [ 0.0666, -0.6049]]]])
torch.Size([1, 12])
torch.Size([1, 4])
tensor([[1., 1., 1., 1.]])
tensor([[-0.2636,  0.4540,  0.4969, -0.7056, -0.7837,  1.8423,  0.5035,  0.3321,
         -0.4469, -0.0372,  0.0666, -0.6049]])
tensor([[-0.2636, -0.2636, -0.2636, -0.2636],
        [ 0.4540,  0.4540,  0.4540,  0.4540],
        [ 0.4969,  0.4969,  0.4969,  0.4969],
        [-0.7056, -0.7056, -0.7056, -0.7056],
        [-0.7837, -0.7837, -0.7837, -0.7837],
        [ 1.8423,  1.8423,  1.8423,  1.8423],
        [ 0.5035,  0.5035,  0.5035,  0.5035],
        [ 0.3321,  0.3321,  0.3321,  0.3321],
        [-0.4469, -0.4469, -0.4469, -0.4469],
        [-0.0372, -0.0372, -0.0372, -0.0372],
        [ 0.0666,  0.0666,  0.0666,  0.0666],
        [-0.6049, -0.6049, -0.6049, -0.6049]])
torch.Size([1, 3, 4, 4]) torch.Size([1, 3, 4, 4])
tensor([[[[-0.26

In [58]:
inp = torch.randn(1, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (5, 5))
print(inp_unf.shape)

torch.Size([1, 75, 48])


In [96]:
def upsample(a,factor_size):
    
    b = torch.ones(factor_size,factor_size)
    print(b)
    a0, a1, s1, s2 = a.shape
    s3, s4 = b.shape
    a = a.reshape(a0, a1, s1, 1, s2, 1)
    b = b.reshape(1, s3, 1, s4)
    return (a * b).reshape(a0, a1, s1 * s3, s2 * s4)

a = torch.randint(10,(1,3,2,2))


print(a)

c = upsample(a,2)
print(c.shape)
print(c)

tensor([[[[0, 1],
          [5, 9]],

         [[8, 5],
          [8, 3]],

         [[8, 1],
          [7, 2]]]])
tensor([[1., 1.],
        [1., 1.]])
torch.Size([1, 3, 4, 4])
tensor([[[[0., 0., 1., 1.],
          [0., 0., 1., 1.],
          [5., 5., 9., 9.],
          [5., 5., 9., 9.]],

         [[8., 8., 5., 5.],
          [8., 8., 5., 5.],
          [8., 8., 3., 3.],
          [8., 8., 3., 3.]],

         [[8., 8., 1., 1.],
          [8., 8., 1., 1.],
          [7., 7., 2., 2.],
          [7., 7., 2., 2.]]]])


In [57]:
def backward_pass(w1, b1, w2, b2,
                  t,
                  x, s1, x1, s2, x2,
                  dl_dw1, dl_db1, dl_dw2, dl_db2):

SyntaxError: unexpected EOF while parsing (<ipython-input-57-bf3db8bb2963>, line 4)

In [ ]:
a = torch.randint(10,(1,10,6,6))
print(a.shape)

upsample = Upsample(3)
upsample2 = Upsample(2)
convSample1 = Convolution(10, 10, kernel_size = 2, stride = 1)
convSample2 = Convolution(10, 3, kernel_size = 3, stride = 1)


c = upsample.forward(a)
print('c',c.shape)
c = convSample1.forward(c)
print('c',c.shape)
c = upsample2.forward(c)
print('c',c.shape)
c = convSample2.forward(c)
print('c',c.shape)